# Configuration


1.   importing librarys
2.   checking gpu
3.   mounting google drive





In [2]:
# importing all librarys
from google.colab import drive
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import PIL.ImageShow
import time
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt



In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Aug 13 04:09:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# run only if using gpu!!
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load

In [13]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = '/content/drive/MyDrive/Articles_Seperation_DL/Split_Data/Test'
train_dataset = tf.keras.utils.image_dataset_from_directory(directory,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE,
                                                          subset='training',
                                                          shuffle=True,
                                                          seed=42,
                                                          validation_split=0.2)
validation_dataset = tf.keras.utils.image_dataset_from_directory(directory,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             subset='validation',
                                             validation_split=0.2,
                                             seed=42,
                                             shuffle=True)

Found 1024 files belonging to 2 classes.
Using 820 files for training.
Found 1024 files belonging to 2 classes.
Using 204 files for validation.


In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [15]:
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [16]:
print(train_dataset)
print(validation_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
class_names = train_dataset.class_names
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

['first_pages', 'other_pages']


# Loading Data with Generator


In [6]:
# create a data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [7]:
train_it = datagen.flow_from_directory('/content/drive/MyDrive/Articles_Seperation_DL/Split_Data/Train/',
                                       class_mode='binary',target_size=(224, 224),
                                       batch_size=32, color_mode="grayscale")

Found 5056 images belonging to 2 classes.


In [8]:
test_it =datagen.flow_from_directory('/content/drive/MyDrive/Articles_Seperation_DL/Split_Data/Test/',
                                     class_mode='binary',target_size=(224, 224),
                                     batch_size=32, color_mode="grayscale")

Found 1024 images belonging to 2 classes.


# Model


In [17]:
# Define the model
model = tf.keras.Sequential()

# Ad a resize layer
# model.add(layers.)

# Add a convolutional layer with 64 filters, a 3x3 kernel, and 'relu' activation
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))

# Add a max pooling layer with 2x2 pool size
model.add(layers.MaxPooling2D((2, 2)))

# Add another convolutional layer with 64 filters and 'relu' activation
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten the output from the previous layer
model.add(layers.Flatten())

# Add a dense (fully connected) layer with 32 units and 'relu' activation
model.add(layers.Dense(32, activation='relu'))

# Add an output layer with 1 unit and 'sigmoid' activation for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 32

In [11]:
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 224, 224, 1), min=0.000, max=1.000


In [19]:
# Train the model
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=10)

# Evaluate the model on the test data
# loss = model.evaluate_generator(test_it, steps=16)

# Print the model summary
print(model.summary())

Epoch 1/10
26/26 [==============================] - 293s 10s/step - loss: 166.5898 - accuracy: 0.6634 - val_loss: 1.1068 - val_accuracy: 0.8578
Epoch 2/10
26/26 [==============================] - 3s 106ms/step - loss: 0.6700 - accuracy: 0.8756 - val_loss: 0.8080 - val_accuracy: 0.8627
Epoch 3/10
26/26 [==============================] - 2s 80ms/step - loss: 0.6588 - accuracy: 0.8768 - val_loss: 0.6424 - val_accuracy: 0.8627
Epoch 4/10
26/26 [==============================] - 3s 85ms/step - loss: 0.6255 - accuracy: 0.8793 - val_loss: 0.6023 - val_accuracy: 0.8480
Epoch 5/10
26/26 [==============================] - 2s 82ms/step - loss: 0.5619 - accuracy: 0.8720 - val_loss: 0.6007 - val_accuracy: 0.8627
Epoch 6/10
26/26 [==============================] - 3s 101ms/step - loss: 0.5846 - accuracy: 0.8780 - val_loss: 0.5803 - val_accuracy: 0.8627
Epoch 7/10
26/26 [==============================] - 3s 104ms/step - loss: 0.5635 - accuracy: 0.8780 - val_loss: 0.5609 - val_accuracy: 0.8627
Epoch 8

In [ ]:
# Save the model to h5 file
model.save('/content/drive/MyDrive/Articles_Seperation_DL/model_5.h5')

In [ ]:
# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/Articles_Seperation_DL/model_5.h5')

In [ ]:
y_test_pred = model.predict(train_dataset).reshape(-1)


NameError: ignored

In [ ]:
y_test_pred_classes = (y_test_pred > 0.5)

In [ ]:
mistakes = np.where(y_test != y_test_pred_classes)[0]

In [ ]:
mistakes_images = X_test[mistakes, :, :, :]

In [ ]:
print(mistakes_images.shape)

(32, 224, 224, 1)


In [ ]:
tru_labels_of_mistakes = y_test[mistakes]

In [ ]:
print()
img = Image.fromarray(mistakes_images[1].reshape(224, 224), mode='L')
img.show()

NameError: ignored

In [ ]:
for i in range(len(mistakes)):
    img = Image.fromarray(mistakes_images[i].reshape(224, 224) * 255, mode='L')
    img.save('/content/drive/MyDrive/Articles_Seperation_DL/'
     + str(i) + 'true label' + str(tru_labels_of_mistakes[i]) + '.png')

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
